In [1]:
import numpy as np
import pandas as pd
from pandas.tseries.offsets import MonthEnd
from pathlib import Path
import statsmodels.api as sm

from stargazer.stargazer import Stargazer

In [2]:
ls1 = ["Mkt-RFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF"]
ls2 = ["Mkt-RF", "MOM"]
ls3 = ["Mkt-RF", "MOM", "VIX"]
ls4 = ["Mkt-RF", "MOM", "VIX", "VVIX"]

In [3]:
a = "string"

In [4]:
ls = [ls1, ls2, ls3, ls4]

In [5]:
max(ls, key= lambda ls: len(ls))

['Mkt-RF', 'MOM', 'VIX', 'VVIX']

In [48]:
ls = ["MOM_RET", "CAPM_MOM", "CAPM_SMB"]

In [60]:
[i.replace("_", "\_") for i in ls]

['MOM\\_RET', 'CAPM\\_MOM', 'CAPM\\_SMB']

In [62]:
dc = {"a": 1, "b": 2}

In [63]:
dc.get("a")

1

In [8]:
def load_pfret_from_pfs(path_portfolios: Path) -> pd.DataFrame:
    """Load the 'option_ret' column of each (class and long-short) portfolio saved 
    in the 'path_portfolios' folder and concatenate to a dataframe.
        
        Args:
            path_data (Path):          Path where the portfolio .csv files reside.   

        Returns:
            pf_returns (pd.DataFrame): All option returns in columns in a dataframe.
    """
    pf_returns = []
    for file in Path.iterdir(path_portfolios):
        try:
            df = pd.read_csv(path_portfolios/file, parse_dates=["date"], index_col="date")
        except PermissionError as err:
            raise PermissionError("The 'portfolios' subfolder must not contain directories."
                                  "Please move or delete the subdirectory.") from err
            # from err necessary for chaining
        pf_returns.append(df["option_ret"].rename(file.name[:-4])) #rename Series to filename from portfolios.
    # Sort list in descending order first -> class0, class1, ..., etc.
    pf_returns = sorted(pf_returns, key=lambda x: x.name)
    pf_returns = pd.concat(pf_returns, axis=1) # Series names -> column names
    # Capitalize 'date' index name for plot axis label.
    pf_returns.index = pf_returns.index.rename("Date")
    return pf_returns

In [9]:
def load_ff_monthly(path_data: Path) -> pd.DataFrame:
    """Load the monthly data of the 5 Factors of Fama and French: Mkt-RF, SMB,
    HML, RMW, CMA and adjust dates to the true end of month dates.
        
        Args:
            path_data (Path):          Parent path where the Fama French data resides.   

        Returns:
            ff_monthly (pd.DataFrame): All factors in columns in a dataframe.
    """
    ff_monthly = pd.read_csv(path_data/"F-F_Research_Data_5_Factors_2x3.csv", skiprows=2).iloc[:706]
    # Rename Unnamed: 0 to Date and convert to appropriate Datetime format.
    ff_monthly = ff_monthly.rename(columns={"Unnamed: 0": "Date"})
    ff_monthly["Date"] = pd.to_datetime(ff_monthly["Date"], format="%Y%m") + MonthEnd(0)
    ff_monthly = ff_monthly.set_index("Date")
    # Convert ff columns to float (have been read as strings).
    for col in ff_monthly.columns:
        ff_monthly[col] = ff_monthly[col].astype("float")
    # Divide by 100 (numbers are in percent).
    ff_monthly =  ff_monthly / 100
    # Drop riskfree rate column (will not be used as a factor in the regressions).
    ff_monthly = ff_monthly.drop(columns="RF")
    return ff_monthly


In [10]:
def filter_idx(df: pd.DataFrame, df_target: pd.DataFrame) -> pd.DataFrame:
    """Filters indeces of 'df' to align with index of 'df_target'.
    
    Returns: Filtered DataFrame.
    """
    return df.loc[df.index[np.isin(df.index, df_target.index)]]

In [11]:
exp_path = Path(r"C:\Users\Mathiass\Documents\Projects\master-thesis\logs\tune\nn_loops\20220909134900")
path_portfolios = exp_path/"portfolios"
path_data = Path(r"C:\Users\Mathiass\Documents\Projects\master-thesis\data")

In [12]:
pf_returns = load_pfret_from_pfs(path_portfolios)
ff_monthly = load_ff_monthly(path_data)

In [13]:
ff_monthly

,Mkt-RF,SMB,HML,RMW,CMA
Date,,,,,
1963-07-31,-0.0039,-0.0044,-0.0089,0.0068,-0.0123
1963-08-31,0.0507,-0.0075,0.0168,0.0036,-0.0034
1963-09-30,-0.0157,-0.0055,0.0008,-0.0071,0.0029
1963-10-31,0.0253,-0.0137,-0.0014,0.0280,-0.0202
1963-11-30,-0.0085,-0.0089,0.0181,-0.0051,0.0231
...,...,...,...,...,...
2021-12-31,0.0310,-0.0069,0.0322,0.0475,0.0436
2022-01-31,-0.0625,-0.0395,0.1274,0.0073,0.0773
2022-02-28,-0.0229,0.0290,0.0309,-0.0212,0.0299


In [14]:
# Variable of interest (y). We want to explain the monthly long short PF returns.
y = pf_returns["long4short0"]

In [15]:
ff_monthly = filter_idx(ff_monthly, y)

In [16]:
y

Date
2008-01-31    0.003212
2008-02-29    0.004047
2008-03-31    0.055884
2008-04-30    0.017467
2008-05-31   -0.038243
                ...   
2021-06-30    0.074146
2021-07-31    0.107212
2021-08-31    0.084287
2021-09-30   -0.634902
2021-10-31    0.028390
Name: long4short0, Length: 166, dtype: float64

In [17]:
ff_monthly

,Mkt-RF,SMB,HML,RMW,CMA
Date,,,,,
2008-01-31,-0.0636,-0.0061,0.0401,0.0221,0.0213
2008-02-29,-0.0309,-0.0064,-0.0084,0.0083,-0.0093
2008-03-31,-0.0093,0.0061,0.0035,0.0080,0.0051
2008-04-30,0.0460,-0.0119,-0.0109,0.0168,-0.0246
2008-05-31,0.0186,0.0302,-0.0153,0.0091,-0.0006
...,...,...,...,...,...
2021-06-30,0.0275,-0.0028,-0.0782,-0.0214,-0.0104
2021-07-31,0.0127,-0.0455,-0.0174,0.0537,-0.0054
2021-08-31,0.0291,-0.0078,-0.0015,-0.0026,-0.0166


In [18]:
regression_map = {"3FF":              ["Mkt-RF"],
                   "3FF_SMB":         ["Mkt-RF", "SMB"],
                  "3FF_HML":     ["Mkt-RF", "SMB", "HML"],
                 }

In [19]:
estimates = []
for v in regression_map.values():
    X = sm.add_constant(ff_monthly[v])
    estimates.append(sm.OLS(y, X).fit())

In [20]:
for i in estimates:
    print(i.model.endog_names)

long4short0
long4short0
long4short0


In [21]:
string = "Roger Federer"

In [22]:
ord("e")

101

In [23]:
table = {ord("e"): "a"}

In [24]:
string.translate(table)

'Rogar Fadarar'

In [25]:
estimates

In [26]:
stargazer = Stargazer(estimates)

In [27]:
# cov_order = ["const"] + ["Mkt-RF", "SMB", "HML"]
# stargazer.covariate_order(cov_order)

In [28]:
stargazer.covariate_order(["Mkt-RF", "SMB", "HML"])

In [29]:
# stargazer.custom_columns(['Model 1', "Model 3"], [1, 2])

In [30]:
stargazer

In [31]:
stargazer

In [32]:
file_name = "test.txt" #Include directory path if needed
tex_file = open(file_name, "w") #This will overwrite an existing file
tex_file.write(stargazer.render_latex())
tex_file.close()

In [33]:
print(stargazer.render_latex())

\begin{table}[!htbp] \centering
\begin{tabular}{@{\extracolsep{5pt}}lccc}
\\[-1.8ex]\hline
\hline \\[-1.8ex]
& \multicolumn{3}{c}{\textit{Dependent variable:}} \
\cr \cline{3-4}
\\[-1.8ex] & (1) & (2) & (3) \\
\hline \\[-1.8ex]
 Mkt-RF & 0.107$^{}$ & 0.142$^{}$ & 0.154$^{}$ \\
  & (0.113) & (0.123) & (0.125) \\
 SMB & & -0.154$^{}$ & -0.100$^{}$ \\
  & & (0.217) & (0.229) \\
 HML & & & -0.137$^{}$ \\
  & & & (0.184) \\
\hline \\[-1.8ex]
 Observations & 166 & 166 & 166 \\
 $R^2$ & 0.005 & 0.008 & 0.012 \\
 Adjusted $R^2$ & -0.001 & -0.004 & -0.006 \\
 Residual Std. Error & 0.068(df = 164) & 0.068(df = 163) & 0.068(df = 162)  \\
 F Statistic & 0.898$^{}$ (df = 1.0; 164.0) & 0.698$^{}$ (df = 2.0; 163.0) & 0.649$^{}$ (df = 3.0; 162.0) \\
\hline
\hline \\[-1.8ex]
\textit{Note:} & \multicolumn{3}{r}{$^{*}$p$<$0.1; $^{**}$p$<$0.05; $^{***}$p$<$0.01} \\
\end{tabular}
\end{table}


In [44]:
stargazer_html = stargazer.render_html()

In [45]:
# html_file = open("stargazer.html", "w")
# html_file.write(stargazer_html)
# html_file.close()

In [46]:
# from html2image import Html2Image
# hti = Html2Image()

In [47]:
# # html = """<h1> An interesting title </h1> This page will be red"""
# css = "body {background: white;}"

# hti.screenshot(html_str=stargazer_html, css_str=css, save_as='red_page.png')

['C:\\Users\\Mathiass\\Documents\\Projects\\master-thesis\\notebooks\\red_page.png']